In [2]:
#default_exp models

In [3]:
#hide
from nbdev import *
%load_ext autoreload
%autoreload 2

# Models

> Uniserie models implementations.

In [4]:
#hide
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [5]:
#export
from itertools import count
from numbers import Number
from typing import Collection, List, Optional, Sequence, Tuple

import numpy as np
import pandas as pd
from numba import njit
from scipy.optimize import minimize
from scipy.stats import norm

from statsforecast.arima import auto_arima_f, forecast_arima

In [6]:
#exporti
@njit
def _ses_fcst_mse(x: np.ndarray, alpha: float) -> Tuple[float, float]:
    """Perform simple exponential smoothing on a series.

    This function returns the one step ahead prediction
    as well as the mean squared error of the fit.
    """
    smoothed = x[0]
    n = x.size
    mse = 0.

    for i in range(1, n):
        smoothed = (alpha * x[i - 1] + (1 - alpha) * smoothed).item()
        error = x[i] - smoothed
        mse += error * error

    mse /= n
    forecast = alpha * x[-1] + (1 - alpha) * smoothed
    return forecast, mse


def _ses_mse(alpha: float, x: np.ndarray) -> float:
    """Compute the mean squared error of a simple exponential smoothing fit."""
    _, mse = _ses_fcst_mse(x, alpha)
    return mse


@njit
def _ses_forecast(x: np.ndarray, alpha: float) -> float:
    """One step ahead forecast with simple exponential smoothing."""
    forecast, _ = _ses_fcst_mse(x, alpha)
    return forecast


@njit
def _demand(x: np.ndarray) -> np.ndarray:
    """Extract the positive elements of a vector."""
    return x[x > 0]


@njit
def _intervals(x: np.ndarray) -> np.ndarray:
    """Compute the intervals between non zero elements of a vector."""
    y = []

    ctr = 1
    for val in x:
        if val == 0:
            ctr += 1
        else:
            y.append(ctr)
            ctr = 1

    y = np.array(y)
    return y


@njit
def _probability(x: np.ndarray) -> np.ndarray:
    """Compute the element probabilities of being non zero."""
    return (x != 0).astype(np.int32)


def _optimized_ses_forecast(x: np.ndarray,
                            bounds: Sequence[Tuple[float, float]] = [(0.1, 0.3)]
                            ) -> float:
    """Searches for the optimal alpha and computes SES one step forecast."""
    alpha = minimize(
        fun=_ses_mse,
        x0=(0,),
        args=(x,),
        bounds=bounds,
        method='L-BFGS-B'
    ).x[0]
    forecast = _ses_forecast(x, alpha)
    return forecast


@njit
def _chunk_sums(array: np.ndarray, chunk_size: int) -> np.ndarray:
    """Splits an array into chunks and returns the sum of each chunk."""
    n = array.size
    n_chunks = n // chunk_size
    sums = np.empty(n_chunks)
    for i, start in enumerate(range(0, n, chunk_size)):
        sums[i] = array[start : start + chunk_size].sum()
    return sums

In [31]:
#export
@njit
def ses(X, h, future_xreg, alpha):
    y = X[:, 0] if X.ndim == 2 else X
    fcst, _ = _ses_fcst_mse(y, alpha)
    return np.full(h, fcst, np.float32)


def adida(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return np.repeat(np.float32(0), h)
    y_intervals = _intervals(y)
    mean_interval = y_intervals.mean()
    aggregation_level = round(mean_interval)
    lost_remainder_data = len(y) % aggregation_level
    y_cut = y[lost_remainder_data:]
    aggregation_sums = _chunk_sums(y_cut, aggregation_level)
    sums_forecast = _optimized_ses_forecast(aggregation_sums)
    forecast = sums_forecast / aggregation_level
    return np.repeat(forecast, h)


@njit
def historic_average(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    return np.repeat(y.mean(), h)


@njit
def croston_classic(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    yd = _demand(y)
    yi = _intervals(y)
    ydp = _ses_forecast(yd, 0.1)
    yip = _ses_forecast(yi, 0.1)
    return ydp / yip


@njit
def croston_sba(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    return 0.95 * croston_classic(y, h, future_xreg)


def croston_optimized(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    yd = _demand(y)
    yi = _intervals(y)
    ydp = _optimized_ses_forecast(yd)
    yip = _optimized_ses_forecast(yi)
    return ydp / yip


@njit
def seasonal_window_average(
    X: np.ndarray,
    h: int,
    future_xreg,
    season_length: int,
    window_size: int,
) -> np.ndarray:
    y = X[:, 0] if X.ndim == 2 else X
    min_samples = season_length * window_size
    if y.size < min_samples:
        return np.full(h, np.nan, np.float32)
    season_avgs = np.zeros(season_length, np.float32)
    for i, value in enumerate(y[-min_samples:]):
        season = i % season_length
        season_avgs[season] += value / window_size
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_avgs[i % season_length]
    return out


@njit
def seasonal_naive(X, h, future_xreg, season_length):
    return seasonal_window_average(X, h, future_xreg, season_length, 1)


def imapa(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return np.repeat(np.float32(0), h)
    y_intervals = _intervals(y)
    mean_interval = y_intervals.mean().item()
    max_aggregation_level = round(mean_interval)
    forecasts = np.empty(max_aggregation_level, np.float32)
    for aggregation_level in range(1, max_aggregation_level + 1):
        lost_remainder_data = len(y) % aggregation_level
        y_cut = y[lost_remainder_data:]
        aggregation_sums = _chunk_sums(y_cut, aggregation_level)
        forecast = _optimized_ses_forecast(aggregation_sums)
        forecasts[aggregation_level - 1] = (forecast / aggregation_level)
    forecast = forecasts.mean()
    return np.repeat(forecast, h)


@njit
def _naive(X, h, future_xreg, return_se):
    y = X[:, 0] if X.ndim == 2 else X
    mean = np.repeat(y[-1], h)
    if not return_se:
        return mean, None
    residuals = y - np.roll(y, -1)
    residuals = residuals[1:]
    se = np.sqrt(np.mean(residuals**2))
    se = np.array([se * np.sqrt(h_) for h_ in range(1, h+1)])
    return mean, se


def naive(X, h, future_xreg, level=None):
    if level is None:
        mean, se = _naive(X=X, h=h, future_xreg=future_xreg, return_se=False)
        return mean
    mean, se = _naive(X=X, h=h, future_xreg=future_xreg, return_se=True)
    quantiles = norm.ppf(0.5 * (1 + np.asarray(level) / 100))
    lo = {
        f'lo-{l}%': mean.reshape(-1, 1) - quantiles * se.reshape(-1, 1)
        for l in level
    }
    hi = {
        f'hi-{l}%': mean.reshape(-1, 1) + quantiles * se.reshape(-1, 1)
        for l in level
    }
    return {
        'mean': mean,
        **lo,
        **hi,
    }    


@njit
def random_walk_with_drift(X, h, future_xreg):
    y = X[:, 0] if X.ndim == 2 else X
    slope = (y[-1] - y[0]) / (y.size - 1)
    return slope * (1 + np.arange(h)) + y[-1]


@njit
def window_average(X, h, future_xreg, window_size):
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < window_size:
        return np.full(h, np.nan, np.float32)
    wavg = y[-window_size:].mean()
    return np.repeat(wavg, h)


@njit
def seasonal_exponential_smoothing(X, h, future_xreg, season_length, alpha):
    y = X[:, 0] if X.ndim == 2 else X
    if y.size < season_length:
        return np.full(h, np.nan, np.float32)
    season_vals = np.empty(season_length, np.float32)
    for i in range(season_length):
        season_vals[i] = _ses_forecast(y[i::season_length], alpha)
    out = np.empty(h, np.float32)
    for i in range(h):
        out[i] = season_vals[i % season_length]
    return out


@njit
def tsb(X, h, future_xreg, alpha_d, alpha_p):
    y = X[:, 0] if X.ndim == 2 else X
    if (y == 0).all():
        return np.repeat(np.float32(0), h)
    yd = _demand(y)
    yp = _probability(y)
    ypf = _ses_forecast(yp, alpha_p)
    ydf = _ses_forecast(yd, alpha_d)
    forecast = np.float32(ypf * ydf)
    return np.repeat(forecast, h)

In [32]:
#export
def auto_arima(X: np.ndarray, h: int, future_xreg=None, season_length: int = 1, 
               approximation: bool = False, level: Optional[Tuple[int]] = None) -> np.ndarray:
    y = X[:, 0] if X.ndim == 2 else X
    xreg = X[:, 1:] if (X.ndim == 2 and X.shape[1] > 1) else None
    mod = auto_arima_f(
        y, 
        xreg=xreg,
        period=season_length, 
        approximation=approximation,
        allowmean=False, allowdrift=False #not implemented yet
    )
    fcst = forecast_arima(mod, h, xreg=future_xreg, level=level)
    if level is None:
        return fcst['mean']
    return {
        'mean': fcst['mean'],
        **{f'lo-{l}': fcst['lower'][f'{l}%'] for l in level},
        **{f'hi-{l}': fcst['upper'][f'{l}%'] for l in level},
    }    

In [33]:
from statsforecast.utils import AirPassengers as ap

In [34]:
auto_arima(ap, 12, season_length=12)

array([444.30005077, 418.2100203 , 446.23703401, 488.22892853,
       499.23136059, 562.23063085, 649.23084981, 633.23078411,
       535.23080382, 488.23079791, 417.23079968, 459.23079915])

External regressors

In [27]:
drift = np.arange(1, ap.size + 1)
X = np.vstack([ap, np.log(drift), np.sqrt(drift)]).T

In [28]:
newdrift = np.arange(ap.size + 1, ap.size + 7 + 1).reshape(-1, 1)
newxreg = np.concatenate([np.log(newdrift), np.sqrt(newdrift)], axis=1)

In [29]:
auto_arima(X, 7, future_xreg=newxreg, season_length=12)

array([444.3798589 , 418.34255927, 446.43035833, 488.47964491,
       499.53980088, 562.59593789, 649.65256062])

Confidence intervals

In [35]:
pd.DataFrame(auto_arima(ap, 12, season_length=12, level=(80, 95)))

,mean,lo-80,lo-95,hi-80,hi-95
0,444.300051,429.182317,421.179472,459.417784,467.420629
1,418.210020,399.756915,389.988431,436.663126,446.431610
2,446.237034,424.256469,412.620660,468.217599,479.853408
3,488.228929,463.405840,450.265291,513.052017,526.192566
4,499.231361,471.807503,457.290190,526.655218,541.172531
5,562.230631,532.446502,516.679736,592.014760,607.781525
6,649.230850,617.256215,600.329866,681.205484,698.131833
7,633.230784,599.207510,581.196677,667.254058,685.264891
8,535.230804,499.275106,480.241310,571.186501,590.220298
9,488.230798,450.441457,430.436989,526.020138,546.024607


In [37]:
pd.DataFrame(naive(ap, 12, None, level=(80,90)))

ValueError: Data must be 1-dimensional